In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import os
os.environ['WANDB_DISABLED'] = 'true'
HOME = os.getcwd()
print(HOME)
!pip install ultralytics==8.2.103 -q
from ultralytics import YOLO

from IPython.display import display, Image

from google.colab import drive
drive.mount('/content/drive')


model = YOLO('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/weights/best.pt')


dataset_path1 = '/content/drive/MyDrive/YOLO_dataset/train2/images'
model.train(data='/content/drive/MyDrive/YOLO_dataset/data2.yaml',epochs=50,batch=16,imgsz=640)
metrics = model.val(data='/content/drive/MyDrive/YOLO_dataset/data2.yaml', split='test')
import glob
results = model.predict(source=dataset_path1, save=True,imgsz=640)
result_images = glob.glob('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Predictie/test_predictions/*.jpg')
#Acum vom face preprocesarea si vom testa daca se va schimba ceva in detectie
import cv2
input_folder='/content/drive/MyDrive/YOLO_dataset/test2/images'
output_folder='/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Preprocesarea_Imaginilor'
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (640, 640))
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_img, 100, 230) # 100 - pragul de jos, 230 - pragul de sus
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, edges)


dataset_path2 = '/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Preprocesarea_Imaginilor'
results2 = model.predict(source=dataset_path1, save=True,imgsz=640,project='/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Preprocesarea_Imaginilor', name='test_predictions_with_preprocessing')
result_images2 = glob.glob('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Predictie/test_predictions_with_preprocessing/*.jpg')







/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
New https://pypi.org/project/ultralytics/8.3.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/PROIECT_PI/runs/detect/yolov8n.pt, data=/content/drive/MyDrive/YOLO_dataset/data2.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det

train: Scanning /content/drive/.shortcut-targets-by-id/1UK7V9ixDNMSvOZePWVtDPqYhjrR7jkSr/YOLO_dataset/train2/labels.cache... 1294 images, 8 backgrounds, 0 corrupt: 100%|██████████| 1294/1294 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2, len(boxes) = 1462. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/.shortcut-targets-by-id/1UK7V9ixDNMSvOZePWVtDPqYhjrR7jkSr/YOLO_dataset/valid2/labels.cache... 368 images, 2 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

lr/pg0,▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▃▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▂▃▃▁▅▆▆▅▆▆▇▇▇▇▇▇▇▇▇██▇██████████████████
metrics/mAP50-95(B),▂▂▂▁▃▄▄▅▅▅▆▅▆▅▆▆▇▆▇▆▇▇▇▇▇▇▇▇▇█▇█████████
metrics/precision(B),▂▃▂▁▅▅▆▅▅▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇██▇█▇███████████
metrics/recall(B),▂▂▂▁▄▆▅▅▆▆▆▇▇▇▇▇▇▆▇▇▇█▇██▇███▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,████▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁


Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/.shortcut-targets-by-id/1UK7V9ixDNMSvOZePWVtDPqYhjrR7jkSr/YOLO_dataset/test2/labels.cache... 186 images, 3 backgrounds, 0 corrupt: 100%|██████████| 186/186 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:09<00:00,  1.29it/s]


                   all        186        214      0.931      0.878       0.94      0.712
Speed: 0.8ms preprocess, 7.0ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/train22


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1294 /content/drive/MyDrive/YOLO_dataset/train2/images/100_jpg.rf.5492400de98373c06ddec3144503bcae.jpg: 640x640 1 Trecere de pietoni, 7.3ms
image 2/1294 /content/drive/MyDrive/YOLO_dataset/train2/images/100_jpg.rf.5a87a46179d13e611b812d2535

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Predictie/test_predictions_with_preprocessing/101_jpg.rf.6ca05b896f1b398f59de433aff5bb42f.jpg'